In [ ]:
import sys
!{sys.executable} -m pip install -U duckdb

In [1]:
import duckdb
print(duckdb.__version__)

0.9.1


In [2]:
# connect to chinook.duckdb instance
db = duckdb.connect(database='../data/chinook/duckdb/chinook.duckdb')

In [3]:
# drop tables
db.execute("""
drop table if exists albums;
drop table if exists artists;
drop table if exists customers;
drop table if exists employees;
drop table if exists genres;
drop table if exists invoices;
drop table if exists invoice_items;
drop table if exists media_types;
drop table if exists playlists;
drop table if exists playlist_track;
drop table if exists tracks;
""")

In [4]:
# create tables
db.execute("""
create table albums
(
  album_id integer not null,
  title nvarchar(160) not null,
  artist_id integer not null,
  constraint pk_album primary key (album_id)
);

create table artists
(
  artist_id integer not null,
  name nvarchar(120),
  constraint pk_artist primary key (artist_id)
);

create table customers
(
  customer_id integer not null,
  first_name nvarchar(40) not null,
  last_name nvarchar(20) not null,
  company nvarchar(80),
  address nvarchar(70),
  city nvarchar(40),
  state nvarchar(40),
  country nvarchar(40),
  postal_code nvarchar(10),
  phone nvarchar(24),
  fax nvarchar(24),
  email nvarchar(60) not null,
  support_rep_id integer,
  constraint pk_customer primary key (customer_id)
);

create table employees
(
  employee_id integer not null,
  last_name nvarchar(20) not null,
  first_name nvarchar(20) not null,
  title nvarchar(30),
  reports_to integer,
  birth_date date,
  hire_date date,
  address nvarchar(70),
  city nvarchar(40),
  state nvarchar(40),
  country nvarchar(40),
  postal_code nvarchar(10),
  phone nvarchar(24),
  fax nvarchar(24),
  email nvarchar(60),
  constraint PK_Employee primary key (employee_id)
);

create table genres
(
  genre_id integer not null,
  name nvarchar(120),
  constraint pk_genre primary key (genre_id)
);

create table invoices
(
  invoice_id integer not null,
  customer_id integer not null,
  invoice_date date not null,
  billing_address nvarchar(70),
  billing_city nvarchar(40),
  billing_state nvarchar(40),
  billing_country nvarchar(40),
  billing_postal_code nvarchar(10),
  total numeric(10,2) not null,
  constraint pk_invoice primary key (invoice_id)
);

create table invoice_items
(
  invoice_line_id integer not null,
  invoice_id integer not null,
  track_id integer not null,
  unit_price numeric(10,2) not null,
  quantity integer not null,
  constraint pk_invoice_line primary key (invoice_line_id)
);

create table media_types
(
  media_type_id integer not null,
  name nvarchar(120),
  constraint pk_media_type primary key (media_type_id)
);

create table playlists
(
  playlist_id integer not null,
  name nvarchar(120),
  constraint pk_playlist primary key (playlist_id)
);

create table playlist_track
(
  playlist_id integer not null,
  track_id integer not null,
  constraint pk_playlist_track primary key (playlist_id, track_id)
);

create table tracks
(
  track_id integer not null,
  name nvarchar(200) not null,
  album_id integer,
  media_type_id integer not null,
  genre_id integer,
  composer nvarchar(220),
  milliseconds integer not null,
  bytes integer,
  unit_price numeric(10,2) not null,
  constraint pk_track primary key (track_id)
);
""")

In [5]:
# show created database schema
db.execute('select * from information_schema.schemata')
db.fetch_df()

,catalog_name,schema_name,schema_owner,default_character_set_catalog,default_character_set_schema,default_character_set_name,sql_path
0,chinook,information_schema,duckdb,NaN,NaN,NaN,None
1,chinook,main,duckdb,NaN,NaN,NaN,None
2,chinook,pg_catalog,duckdb,NaN,NaN,NaN,None
3,system,information_schema,duckdb,NaN,NaN,NaN,None
4,system,main,duckdb,NaN,NaN,NaN,None
5,system,pg_catalog,duckdb,NaN,NaN,NaN,None
6,temp,information_schema,duckdb,NaN,NaN,NaN,None
7,temp,main,duckdb,NaN,NaN,NaN,None
8,temp,pg_catalog,duckdb,NaN,NaN,NaN,None


In [6]:
# show created database tables
db.execute('select * from information_schema.tables')
db.fetch_df()

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action
0,chinook,main,albums,BASE TABLE,NaN,NaN,NaN,NaN,NaN,YES,NO,None
1,chinook,main,artists,BASE TABLE,NaN,NaN,NaN,NaN,NaN,YES,NO,None
2,chinook,main,customers,BASE TABLE,NaN,NaN,NaN,NaN,NaN,YES,NO,None
3,chinook,main,employees,BASE TABLE,NaN,NaN,NaN,NaN,NaN,YES,NO,None
4,chinook,main,genres,BASE TABLE,NaN,NaN,NaN,NaN,NaN,YES,NO,None
5,chinook,main,invoices,BASE TABLE,NaN,NaN,NaN,NaN,NaN,YES,NO,None
6,chinook,main,invoice_items,BASE TABLE,NaN,NaN,NaN,NaN,NaN,YES,NO,None
7,chinook,main,media_types,BASE TABLE,NaN,NaN,NaN,NaN,NaN,YES,NO,None
8,chinook,main,playlists,BASE TABLE,NaN,NaN,NaN,NaN,NaN,YES,NO,None
9,chinook,main,playlist_track,BASE TABLE,NaN,NaN,NaN,NaN,NaN,YES,NO,None


In [7]:
# show all table columns
db.execute('select * from information_schema.columns')
db.fetch_df()

,table_catalog,table_schema,table_name,column_name,ordinal_position,column_default,is_nullable,data_type,character_maximum_length,character_octet_length,...,is_identity,identity_generation,identity_start,identity_increment,identity_maximum,identity_minimum,identity_cycle,is_generated,generation_expression,is_updatable
0,chinook,main,albums,album_id,1,None,NO,INTEGER,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chinook,main,albums,title,2,None,NO,VARCHAR,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chinook,main,albums,artist_id,3,None,NO,INTEGER,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,chinook,main,artists,artist_id,1,None,NO,INTEGER,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,chinook,main,artists,name,2,None,YES,VARCHAR,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,chinook,main,tracks,genre_id,5,None,YES,INTEGER,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,chinook,main,tracks,composer,6,None,YES,VARCHAR,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,chinook,main,tracks,milliseconds,7,None,NO,INTEGER,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,chinook,main,tracks,bytes,8,None,YES,INTEGER,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# create foreign key indexes
db.execute("""
create index ifk_album_artist_id on albums (artist_id);
create index ifk_customer_support_rep_id on customers (support_rep_id);
create index ifk_employee_reports_to on employees (reports_to);
create index ifk_invoice_customer_id on invoices (customer_id);
create index ifk_invoice_item_invoice_id on invoice_items (invoice_id);
create index ifk_invoice_item_track_id on invoice_items (track_id);
create index ifk_playlist_track_track_id on playlist_track (track_id);
create index ifk_track_album_id on tracks (album_id);
create index ifk_track_genre_id on tracks (genre_id);
create index ifk_track_media_type_id on tracks (media_type_id);
""")

In [9]:
# load csv data
db.execute("""
insert into artists select * from '../data/chinook/csv/artists.csv';
insert into albums select * from '../data/chinook/csv/albums.csv';
insert into genres select * from '../data/chinook/csv/genres.csv';
insert into media_types select * from '../data/chinook/csv/media_types.csv';
insert into playlists select * from '../data/chinook/csv/playlists.csv';
insert into tracks select * from '../data/chinook/csv/tracks.csv';
insert into playlist_track select * from '../data/chinook/csv/playlist_track.csv';
insert into employees select * from '../data/chinook/csv/employees.csv';
insert into customers select * from '../data/chinook/csv/customers.csv';
insert into invoices select * from '../data/chinook/csv/invoices.csv';
insert into invoice_items select * from '../data/chinook/csv/invoice_items.csv';
""")


In [10]:
db.execute('select * from artists limit 10')
print(db.fetch_df())

   artist_id                  name
0          1                 AC/DC
1          2                Accept
2          3             Aerosmith
3          4     Alanis Morissette
4          5       Alice In Chains
5          6  Antônio Carlos Jobim
6          7          Apocalyptica
7          8            Audioslave
8          9              BackBeat
9         10          Billy Cobham


In [11]:
db.execute('pragma show_tables')
print(db.fetch_df())

              name
0           albums
1          artists
2        customers
3        employees
4           genres
5    invoice_items
6         invoices
7      media_types
8   playlist_track
9        playlists
10          tracks


In [12]:
db.execute('select * from information_schema.columns')
print(db.fetch_df())

   table_catalog table_schema table_name   column_name  ordinal_position  \
0        chinook         main     albums      album_id                 1   
1        chinook         main     albums         title                 2   
2        chinook         main     albums     artist_id                 3   
3        chinook         main    artists     artist_id                 1   
4        chinook         main    artists          name                 2   
..           ...          ...        ...           ...               ...   
59       chinook         main     tracks      genre_id                 5   
60       chinook         main     tracks      composer                 6   
61       chinook         main     tracks  milliseconds                 7   
62       chinook         main     tracks         bytes                 8   
63       chinook         main     tracks    unit_price                 9   

   column_default is_nullable      data_type  character_maximum_length  \
0            

In [ ]:
# export created chinook.duckdb with data
db.execute("export database '../data/chinook/duckdb/data'")

In [ ]:
# export chinook data in parquet format
db.execute("""
copy artists to '../data/chinook/duckdb/data/artists.parquet';
copy albums to '../data/chinook/duckdb/data/albums.parquet';
copy genres to '../data/chinook/duckdb/data/genres.parquet';
copy media_types to '../data/chinook/duckdb/data/media_types.parquet';
copy playlists to '../data/chinook/duckdb/data/playlists.parquet';
copy tracks to '../data/chinook/duckdb/data/tracks.parquet';
copy playlist_track to '../data/chinook/duckdb/data/playlist_track.parquet';
copy employees to '../data/chinook/duckdb/data/employees.parquet';
copy customers to '../data/chinook/duckdb/data/customers.parquet';
copy invoices to '../data/chinook/duckdb/data/invoices.parquet';
copy invoice_items to '../data/chinook/duckdb/data/invoice_items.parquet';
""")

In [13]:
# close created chinook.duckdb to save it
db.close()